In [1]:

import pandas as pd
import numpy as np
import pickle
from bs4 import BeautifulSoup
from bs4 import NavigableString

from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn import preprocessing
from sklearn import model_selection 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

## Import Data

In [2]:
# Load necessary data
data_dir = 'mens-machine-learning-competition-2019/DataFiles/'
df_seeds = pd.read_csv(data_dir + 'NCAATourneySeeds.csv')
df_tour = pd.read_csv(data_dir + 'NCAATourneyCompactResults.csv')
df_regular_season = pd.read_csv(data_dir + 'RegularSeasonCompactResults.csv')
df_conference_games = pd.read_csv(data_dir + 'ConferenceTourneyGames.csv')
df_team_rpis = pd.read_csv("RPI/TeamRPI.csv") # scrapped from the web
df_teams = pd.read_csv(data_dir + "Teams.csv")

In [3]:
# collect win counts for each team per season
df_regular_season.drop(labels=['DayNum', 'WScore', 'LScore', 'WLoc', 'NumOT'], inplace=True, axis=1)
df_win_counts = df_regular_season.groupby(['Season', 'WTeamID']).count().reset_index()
df_win_counts = df_win_counts.rename(columns = {'LTeamID': 'RWinCount', 'WTeamID': 'TeamID'})

In [4]:
# collectin loss counts for each team per son
df_loss_counts = df_regular_season.groupby(['Season', 'LTeamID']).count().reset_index()
df_loss_counts = df_loss_counts.rename(columns = {'WTeamID': 'RLossCount', 'LTeamID': 'TeamID'})

In [5]:
# merge win/loss counts and show regular season win percentage
df_regular_season_totals = pd.merge(left=df_win_counts, right=df_loss_counts, how='left', on=['Season', 'TeamID'])
df_regular_season_totals['RWinPerc'] = df_regular_season_totals.RWinCount / (df_regular_season_totals.RWinCount + df_regular_season_totals.RLossCount)
df_regular_season_totals.head()

,Season,TeamID,RWinCount,RLossCount,RWinPerc
0,1985,1102,5,19.0,0.208333
1,1985,1103,9,14.0,0.391304
2,1985,1104,21,9.0,0.700000
3,1985,1106,10,14.0,0.416667
4,1985,1108,19,6.0,0.760000


In [6]:
# drop unused labels
df_conference_games.drop(labels=['ConfAbbrev', 'DayNum'], inplace=True, axis=1)

In [7]:
# gather win/loss totals for conference tournaments and win %
df_c_win_counts = df_conference_games.groupby(['Season', 'WTeamID']).count().reset_index()
df_c_win_counts = df_c_win_counts.rename(columns = {'LTeamID': 'CWinCount', 'WTeamID': 'TeamID'})
df_c_loss_counts = df_conference_games.groupby(['Season', 'LTeamID']).count().reset_index()
df_c_loss_counts = df_c_loss_counts.rename(columns = {'WTeamID': 'CLossCount', 'LTeamID': 'TeamID'})
df_conference_tournament_totals = pd.merge(left=df_c_win_counts, right=df_c_loss_counts, how='left', on=['Season', 'TeamID'])
df_conference_tournament_totals = df_conference_tournament_totals.fillna(value={'CWinCount': 0, 'CLossCount': 0})
df_conference_tournament_totals['CWinPerc'] = df_conference_tournament_totals.CWinCount / (df_conference_tournament_totals.CWinCount + df_conference_tournament_totals.CLossCount)
df_conference_tournament_totals.head()

,Season,TeamID,CWinCount,CLossCount,CWinPerc
0,2001,1104,1,1.0,0.500000
1,2001,1106,3,0.0,1.000000
2,2001,1108,2,1.0,0.666667
3,2001,1111,1,1.0,0.500000
4,2001,1114,1,1.0,0.500000


In [8]:
# combine regular season and conference tournament win/loss
df_win_totals = pd.merge(left=df_regular_season_totals, right=df_conference_tournament_totals, how='left', on=['Season', 'TeamID'])
df_win_totals.tail(10)

,Season,TeamID,RWinCount,RLossCount,RWinPerc,CWinCount,CLossCount,CWinPerc
10864,2018,1455,25,7.0,0.781250,1.0,1.0,0.5
10865,2018,1456,17,12.0,0.586207,1.0,1.0,0.5
10866,2018,1457,16,12.0,0.571429,1.0,1.0,0.5
10867,2018,1458,15,18.0,0.454545,1.0,1.0,0.5
10868,2018,1459,18,12.0,0.600000,1.0,1.0,0.5
10869,2018,1460,23,9.0,0.718750,3.0,0.0,1.0
10870,2018,1461,19,13.0,0.593750,1.0,1.0,0.5
10871,2018,1462,28,5.0,0.848485,1.0,1.0,0.5
10872,2018,1463,15,15.0,0.500000,NaN,NaN,NaN
10873,2018,1464,6,24.0,0.200000,NaN,NaN,NaN


In [9]:
# convert seed into int
df_seeds['Seed_Int'] = df_seeds['Seed'].str[1:3]
df_seeds['Seed_Int'] = df_seeds['Seed_Int'].apply(pd.to_numeric)
# Drop unused labels
df_seeds.drop(labels=['Seed'], inplace=True, axis=1)
df_tour.drop(labels=['DayNum', 'WScore', 'LScore', 'WLoc', 'NumOT'], inplace=True, axis=1)
# Rename to winning team
df_winseeds = df_seeds.rename(columns={'TeamID': 'WTeamID', 'Seed_Int': 'WSeed'})
# Rename lossing teams
df_losseeds = df_seeds.rename(columns={'TeamID': 'LTeamID', 'Seed_Int': 'LSeed'})

In [10]:
df_seeds.head()

,Season,TeamID,Seed_Int
0,1985,1207,1
1,1985,1210,2
2,1985,1228,3
3,1985,1260,4
4,1985,1374,5


In [11]:
df_tour.head()

,Season,WTeamID,LTeamID
0,1985,1116,1234
1,1985,1120,1345
2,1985,1207,1250
3,1985,1229,1425
4,1985,1242,1325


In [12]:
df_winseeds.head()

,Season,WTeamID,WSeed
0,1985,1207,1
1,1985,1210,2
2,1985,1228,3
3,1985,1260,4
4,1985,1374,5


In [13]:
df_losseeds.head()

,Season,LTeamID,LSeed
0,1985,1207,1
1,1985,1210,2
2,1985,1228,3
3,1985,1260,4
4,1985,1374,5


In [15]:
df_dummy = pd.merge(left=df_tour, right=df_winseeds, how='left', on=['Season', 'WTeamID'])
df_concat = pd.merge(left=df_dummy, right=df_losseeds, how='left', on=['Season', 'LTeamID'])
df_concat['SeedDiff'] = df_concat.WSeed - df_concat.LSeed

In [16]:
df_concat.head()

,Season,WTeamID,LTeamID,WSeed,LSeed,SeedDiff
0,1985,1116,1234,9,8,1
1,1985,1120,1345,11,6,5
2,1985,1207,1250,1,16,-15
3,1985,1229,1425,9,8,1
4,1985,1242,1325,3,14,-11


In [58]:
df_team_rpis_concat = pd.merge(left=df_team_rpis, right=df_teams, how="left", on=["TeamName"])
df_team_rpis_concat.dropna()
df_team_rpis_concat.head()

,Unnamed: 0,Season,TeamName,RPI,TeamID,FirstD1Season,LastD1Season
0,0,1999,Duke,0.670,1181.0,1985.0,2019.0
1,1,1999,Kansas,0.657,1242.0,1985.0,2019.0
2,2,1999,Virginia,0.655,1438.0,1985.0,2019.0
3,3,1999,North Carolina,0.653,1314.0,1985.0,2019.0
4,4,1999,Houston,0.648,1222.0,1985.0,2019.0
